Release VERSION 230109

<h1>RSoXS plan simulator</h1>
this is a full stack of the SST scan system, from the loader through to the low level acquisition code, all simulated here
run all the hidden cells first!

In [1]:
%load_ext autoreload
%autoreload 2

from rsoxs_scans.spreadsheets import load_samplesxlsx, save_samplesxlsx
from rsoxs_scans.acquisition import dryrun_bar, get_acq_details

In [2]:
# test your .xlsx file here
# load the example bar
bar = load_samplesxlsx(r"Sample_Bar_v2023_1.NIST.230301Draftv2_Clean.xlsx")

spreadsheet version is 2023-1 Version 1.0
Reading sheet Bar
Reading sheet Acquisitions


In [3]:
save_samplesxlsx(bar,'test2') # test writing to a temporary file

In [4]:
bar2 = load_samplesxlsx("test2.xlsx") # test loading from a temporary file

spreadsheet version is 2023-1 Version 1.0
Reading sheet Bar
Reading sheet Acquisitions


In [3]:
# dry run it
outputs = dryrun_bar(bar,['apriority'],group='all')

/nsls2/users/bpatel/BP_LocalCodeBase/rsoxs_scans/rsoxs_scans/acquisition.py:269: UserWarning: WARNING: acquisition # 38 will take 42.25 minutes, which is more than 30.0 minutes
  warnings.warn(
/nsls2/users/bpatel/BP_LocalCodeBase/rsoxs_scans/rsoxs_scans/acquisition.py:269: UserWarning: WARNING: acquisition # 39 will take 192.55 minutes, which is more than 30.0 minutes
  warnings.warn(
/nsls2/users/bpatel/BP_LocalCodeBase/rsoxs_scans/rsoxs_scans/acquisition.py:269: UserWarning: WARNING: acquisition # 41 will take 122.0 minutes, which is more than 30.0 minutes
  warnings.warn(
/nsls2/users/bpatel/BP_LocalCodeBase/rsoxs_scans/rsoxs_scans/acquisition.py:269: UserWarning: WARNING: acquisition # 44 will take 42.0 minutes, which is more than 30.0 minutes
  warnings.warn(


________________________________________________
Acquisition # 0 from sample p3ht, group 

Summary: load p3ht from testing, config SAXS, run rsoxs priority( sample 1 acquisition 1), starts @ 0:00:00 takes 0:10:00
 (+120 seconds for configuration change)

>Step 0: load configuration SAXS
>Step 1: load sample p3ht
>Step 2: set Diode range to high
>Step 3: setting polarization to 0
 RSoXS scanning ['saxs_det'] from 250.0 eV to 350.0 eV on the rsoxs l/mm grating
    in 114 steps with exposure times from 1.0 to 1.0 seconds

________________________________________________
________________________________________________
Acquisition # 1 from sample pcbm, group 

Summary: load pcbm from testing, config WAXS, run nexafs priority( sample 2 acquisition 2), starts @ 0:12:00 takes 0:03:10
 (+120 seconds for configuration change)

>Step 0: load configuration WAXS
>Step 1: load sample pcbm
>Step 2: set Diode range to high
>Step 3: setting polarization to 0
 setting grating to rsoxs
 fly nexafs scann

In [ ]:
# Get more details on a given acquisition
get_acq_details(40,outputs)

In [ ]:
# get the set of all actions
actions = set()
for acq in outputs:
    for step in acq['steps']:
        actions.add(step['action'])
print(actions)

In [ ]:
arguments = {}
for action in actions:
    keys = set()
    for acq in outputs:
        for step in acq['steps']:
            if step['action'] == action:
                if 'kwargs' in step:
                    for kwarg in step['kwargs']:
                        keys.add(kwarg)
    arguments[action]=keys
    print("{:<18} : {}".format(action,keys))

<h1>Examples of edges, ratios, frames

In [ ]:
# totally manual simple scan
from rsoxs_scans.constructor import get_energies
energies = get_energies((1850,1930),100)

In [ ]:
# precise energy list
# you can fool the algorithm if you put 1 as the number of frames it will only take one frame at each of the thresholds, including the last one, so you can sort of treat this as just an energy list
# as a corollary, you can't have a total number of frames less than the thresholds that you give it in edges (it will at a minimum always take the threshold values)
energies = get_energies((250,250,250,340,340,341,280,281.45,500,500),1)

In [ ]:
# use a default
energies = get_energies('Nitrogen')

In [ ]:
# use a default with a different name
energies = get_energies('c')

In [ ]:
# use a different built in interval set
energies = get_energies('c','full','carbon nonaromatic')

In [ ]:
# get a very short version of this scan
energies = get_energies('c','very short','carbon nonaromatic')

In [ ]:
# I don't have time for this, I only want ~10 energies
energies = get_energies('c',10,'carbon nonaromatic')

In [ ]:
# also, I made single entry energies also work
energies = get_energies(280)

In [ ]:
# also option to have frames as a list of exact numbers needed in each region
energies = get_energies('c',[2,5,10,40,5,2])

In [ ]:
# putting in zero effectively removes that region
energies = get_energies('c',[2,5,10,0,5,2])

In [ ]:
from rsoxs_scans.constructor import construct_exposure_times
times,_ = construct_exposure_times(get_energies([270,280,290,400],500,[2,1,10]),1)
len(times)

In [ ]:
from rsoxs_scans.constructor import get_nexafs_scan_params
get_nexafs_scan_params('n','normal')

<h1>Example of complex Exposure times

In [ ]:
construct_exposure_times(get_energies((1850,1930),100),[2,("between",1870,1900.0),4,("greater_than",1920.0),1],80)

In [ ]:
isinstance('asdf',str)